<a href="https://colab.research.google.com/github/jaredteoh/Singapore-HDB-Resale-Price-Prediction/blob/main/HDB_Resale_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

The main objective of this project is to make sensible predictions of the resale prices of public housing in Singapore (using the dataset obtained from data.gov.sg). Since the location of housing (e.g., distance from the nearest mrt) is important on determining resale prices, we will derive some additional features that will aid our prediction accuracy, as we will see later.

Credits: https://medium.com/@michael.wy.ong/web-scrape-geospatial-data-analyse-singapores-property-price-part-i-276caba320b

In [2]:
import numpy as np
import pandas as pd
import requests
import math
import json
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Web-scraping Data

The resale flat prices can be found on the data.gov.sg website. In this case, we will be using the prices from January 2017 onwards (which has been downloaded beforehand):

In [4]:
resale_price_path = '/content/drive/MyDrive/Colab Notebooks/Housing/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv'
df_resale = pd.read_csv(resale_price_path)
df_resale.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


In [5]:
df_resale['address'] = df_resale['block'] + ' ' + df_resale['street_name']
df_resale.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,address
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,406 ANG MO KIO AVE 10
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,108 ANG MO KIO AVE 4
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,602 ANG MO KIO AVE 5
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,465 ANG MO KIO AVE 10
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,601 ANG MO KIO AVE 5


In [6]:
all_address = list(df_resale['address'])
unique_address = list(set(all_address))
print('Total unique addresses out of {} housing transactions: {}'.format(len(df_resale), len(unique_address)))

Total unique addresses out of 105091 housing transactions: 9100


We will be finding the distance between flats and their respective nearest mrt station, but first we will need to find the coordinates of the flats and mrt stations. Hence, we will make use of onemap.sg to find the coordinates (latiude, longitude) and full addresses of flats by using only their block and street name:

In [7]:
filepath = '/content/drive/MyDrive/Colab Notebooks/Housing/'

A function is defined for reusability in the future on other amenities (e.g., nearest school, nearest food court):

In [8]:
def find_postal(lst, filename):
    '''With the block number and street name, get the full address of the hdb flat,
    including the postal code, geogaphical coordinates (lat/long)'''
    
    for index,add in enumerate(lst):
        # Do not need to change the URL
        url= "https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal="+ add        
        print(index,url)
        
        # Retrieve information from website
        response = requests.get(url)
        data = json.loads(response.text) 
    
        temp_df = pd.DataFrame.from_dict(data["results"])
        # The "add" is the address that was used to search in the website
        temp_df["address"] = add
        
        # Create the file with the first row that is read in 
        if index == 0:
            file = temp_df
        else:
            file = file.append(temp_df)
    file.to_csv(filepath + filename + '.csv')

In [12]:
find_postal(lst=unique_address, filename='full_address')

Streaming output truncated to the last 5000 lines.
4100 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=534 BEDOK NTH ST 3
4101 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=496E TAMPINES AVE 9
4102 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=976 JURONG WEST ST 93
4103 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=488 JURONG WEST AVE 1
4104 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=345 BT BATOK ST 34
4105 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=471 JURONG WEST ST 41
4106 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=848 TAMPINES ST 82
4107 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVa

In [9]:
geo = pd.read_csv(filepath + 'full_address.csv')
geo.head()

,Unnamed: 0,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE,LONGTITUDE,address
0,0,359 WOODLANDS AVENUE 5 SINGAPORE 730359,359,WOODLANDS AVENUE 5,NIL,359 WOODLANDS AVENUE 5 SINGAPORE 730359,730359,22519.257542,46197.037659,1.434064,103.784069,103.784069,359 WOODLANDS AVE 5
1,0,THE PREMIERE @ TAMPINES,515A,TAMPINES CENTRAL 7,THE PREMIERE @ TAMPINES,515A TAMPINES CENTRAL 7 THE PREMIERE @ TAMPINE...,521515,39692.471022,37672.128420,1.356966,103.938383,103.938383,515A TAMPINES CTRL 7
2,0,246 SIMEI STREET 5 SINGAPORE 520246,246,SIMEI STREET 5,NIL,246 SIMEI STREET 5 SINGAPORE 520246,520246,41448.971999,36355.663936,1.345059,103.954166,103.954166,246 SIMEI ST 5
3,0,183 JELEBU ROAD SINGAPORE 670183,183,JELEBU ROAD,NIL,183 JELEBU ROAD SINGAPORE 670183,670183,20179.799074,40215.020853,1.379964,103.763048,103.763048,183 JELEBU RD
4,1,PCF SPARKLETOTS PRESCHOOL @ CASHEW 183 (CC),183,JELEBU ROAD,PCF SPARKLETOTS PRESCHOOL @ CASHEW 183 (CC),183 JELEBU ROAD PCF SPARKLETOTS PRESCHOOL @ CA...,670183,20179.799195,40215.019794,1.379964,103.763048,103.763048,183 JELEBU RD


In [10]:
geo.rename(columns={'Unnamed: 0': 'INDICATOR'}, inplace=True)
geo.head()

,INDICATOR,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE,LONGTITUDE,address
0,0,359 WOODLANDS AVENUE 5 SINGAPORE 730359,359,WOODLANDS AVENUE 5,NIL,359 WOODLANDS AVENUE 5 SINGAPORE 730359,730359,22519.257542,46197.037659,1.434064,103.784069,103.784069,359 WOODLANDS AVE 5
1,0,THE PREMIERE @ TAMPINES,515A,TAMPINES CENTRAL 7,THE PREMIERE @ TAMPINES,515A TAMPINES CENTRAL 7 THE PREMIERE @ TAMPINE...,521515,39692.471022,37672.128420,1.356966,103.938383,103.938383,515A TAMPINES CTRL 7
2,0,246 SIMEI STREET 5 SINGAPORE 520246,246,SIMEI STREET 5,NIL,246 SIMEI STREET 5 SINGAPORE 520246,520246,41448.971999,36355.663936,1.345059,103.954166,103.954166,246 SIMEI ST 5
3,0,183 JELEBU ROAD SINGAPORE 670183,183,JELEBU ROAD,NIL,183 JELEBU ROAD SINGAPORE 670183,670183,20179.799074,40215.020853,1.379964,103.763048,103.763048,183 JELEBU RD
4,1,PCF SPARKLETOTS PRESCHOOL @ CASHEW 183 (CC),183,JELEBU ROAD,PCF SPARKLETOTS PRESCHOOL @ CASHEW 183 (CC),183 JELEBU ROAD PCF SPARKLETOTS PRESCHOOL @ CA...,670183,20179.799195,40215.019794,1.379964,103.763048,103.763048,183 JELEBU RD


In [11]:
geo_edited = geo.drop(columns=['SEARCHVAL', 'BLK_NO', 'ROAD_NAME', 'BUILDING', 'LONGTITUDE'])
geo_edited.head()

,INDICATOR,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE,address
0,0,359 WOODLANDS AVENUE 5 SINGAPORE 730359,730359,22519.257542,46197.037659,1.434064,103.784069,359 WOODLANDS AVE 5
1,0,515A TAMPINES CENTRAL 7 THE PREMIERE @ TAMPINE...,521515,39692.471022,37672.128420,1.356966,103.938383,515A TAMPINES CTRL 7
2,0,246 SIMEI STREET 5 SINGAPORE 520246,520246,41448.971999,36355.663936,1.345059,103.954166,246 SIMEI ST 5
3,0,183 JELEBU ROAD SINGAPORE 670183,670183,20179.799074,40215.020853,1.379964,103.763048,183 JELEBU RD
4,1,183 JELEBU ROAD PCF SPARKLETOTS PRESCHOOL @ CA...,670183,20179.799195,40215.019794,1.379964,103.763048,183 JELEBU RD


In [12]:
geo_add_lat_long = geo_edited.copy()
geo_add_lat_long = geo_add_lat_long[geo_add_lat_long['INDICATOR'] == 0]
geo_add_lat_long.drop(columns=['INDICATOR', 'POSTAL', 'X', 'Y', 'address'], inplace=True)
geo_add_lat_long.head()

,ADDRESS,LATITUDE,LONGITUDE
0,359 WOODLANDS AVENUE 5 SINGAPORE 730359,1.434064,103.784069
1,515A TAMPINES CENTRAL 7 THE PREMIERE @ TAMPINE...,1.356966,103.938383
2,246 SIMEI STREET 5 SINGAPORE 520246,1.345059,103.954166
3,183 JELEBU ROAD SINGAPORE 670183,1.379964,103.763048
5,109 BEDOK NORTH ROAD FENGSHAN ESTATE SINGAPORE...,1.332053,103.934935


The list of mrt stations can be found at https://github.com/hxchua/datadoubleconfirm/blob/master/datasets/mrtsg.csv, with their latitude and longitude provided:

In [13]:
mrt_geo = pd.read_csv(filepath + 'mrtsg.csv')
mrt_geo.head()

,OBJECTID,STN_NAME,STN_NO,X,Y,Latitude,Longitude,COLOR
0,12,ADMIRALTY MRT STATION,NS10,24402.1063,46918.1131,1.440585,103.800998,RED
1,16,ALJUNIED MRT STATION,EW9,33518.6049,33190.0020,1.316433,103.882893,GREEN
2,33,ANG MO KIO MRT STATION,NS16,29807.2655,39105.7720,1.369933,103.849553,RED
3,81,BAKAU LRT STATION,SE3,36026.0821,41113.8766,1.388093,103.905418,OTHERS
4,80,BANGKIT LRT STATION,BP9,21248.2460,40220.9693,1.380018,103.772667,OTHERS


In [14]:
mrt_add_lat_long = pd.DataFrame(pd.concat([mrt_geo[column] for column in ['STN_NAME', 'Latitude', 'Longitude']], axis=1))
mrt_add_lat_long.head()

,STN_NAME,Latitude,Longitude
0,ADMIRALTY MRT STATION,1.440585,103.800998
1,ALJUNIED MRT STATION,1.316433,103.882893
2,ANG MO KIO MRT STATION,1.369933,103.849553
3,BAKAU LRT STATION,1.388093,103.905418
4,BANGKIT LRT STATION,1.380018,103.772667


After determining the geographical locations, we will then define a function to find the distance from the nearest mrt stations for each flat. Similarly, it can be use for other amenities:

In [15]:
from geopy.distance import geodesic

def find_nearest(housing, amenity):
    results = {}
    for index, flat in enumerate(housing.iloc[:, 0]):
        flat_loc = (housing.iloc[index, 1], housing.iloc[index, 2])
        flat_amenity = ['', '', 100]
        for ind, eachloc in enumerate(amenity.iloc[:, 0]):
            amenity_loc = (amenity.iloc[ind, 1], amenity.iloc[ind, 2])
            distance = geodesic(flat_loc, amenity_loc)
            if distance < flat_amenity[2]:
                flat_amenity[0] = flat
                flat_amenity[1] = eachloc
                flat_amenity[2] = distance
        results[flat] = (flat_amenity[1], flat_amenity[2])
        print(index, '-', flat, ':', results[flat])
    return results

In [16]:
distance_mrt = find_nearest(geo_add_lat_long, mrt_add_lat_long)

Streaming output truncated to the last 5000 lines.
4078 - 349 CORPORATION DRIVE SINGAPORE 610349 : ('LAKESIDE MRT STATION', Distance(0.5600912246618154))
4079 - 816 JURONG WEST STREET 81 SINGAPORE 640816 : ('PIONEER MRT STATION', Distance(0.8608031170232496))
4080 - 242 PASIR RIS STREET 21 SINGAPORE 510242 : ('PASIR RIS MRT STATION', Distance(1.485253164682238))
4081 - 5 LORONG LEW LIAN LEW LIAN GARDENS SINGAPORE 530005 : ('SERANGOON MRT STATION', Distance(0.3665759766297628))
4082 - 111 COMMONWEALTH CRESCENT COMMONWEALTH NEIGHBOURHOOD POLICE POST SINGAPORE 140111 : ('COMMONWEALTH MRT STATION', Distance(0.4233958140735266))
4083 - 326B ANCHORVALE ROAD ANCHORVALE HORIZON SINGAPORE 542326 : ('FARMWAY LRT STATION', Distance(0.24962663232398263))
4084 - 10 JOO SENG ROAD SINGAPORE 360010 : ('TAI SENG MRT STATION', Distance(0.8965233665909108))
4085 - 370 WOODLANDS AVENUE 1 SINGAPORE 730370 : ('WOODLANDS MRT STATION', Distance(0.3946020731335983))
4086 - 142 MARSILING ROAD SINGAPORE 730142 :

In [20]:
df_distance_mrt = pd.DataFrame(distance_mrt).T.reset_index()
df_distance_mrt

,index,0,1
0,359 WOODLANDS AVENUE 5 SINGAPORE 730359,WOODLANDS MRT STATION,0.4121162706796198 km
1,515A TAMPINES CENTRAL 7 THE PREMIERE @ TAMPINE...,TAMPINES MRT STATION,0.5576863639731215 km
2,246 SIMEI STREET 5 SINGAPORE 520246,SIMEI MRT STATION,0.22495411013969127 km
3,183 JELEBU ROAD SINGAPORE 670183,BUKIT PANJANG MRT STATION,0.19721228372555052 km
4,109 BEDOK NORTH ROAD FENGSHAN ESTATE SINGAPORE...,BEDOK RESERVOIR MRT STATION,0.588027612421757 km
...,...,...,...
9072,222 JURONG EAST STREET 21 SINGAPORE 600222,CHINESE GARDEN MRT STATION,0.38763476453104184 km
9073,93 COMMONWEALTH DRIVE COMMONWEALTH 16 SINGAPOR...,COMMONWEALTH MRT STATION,0.3675665234452749 km
9074,972 HOUGANG STREET 91 SINGAPORE 530972,TONGKANG LRT STATION,1.422161424690531 km
9075,714 CLEMENTI WEST STREET 2 SINGAPORE 120714,CLEMENTI MRT STATION,1.2452723466691529 km


In [21]:
df_distance_mrt = df_distance_mrt.rename(columns={'index':'ADDRESS', 0: 'MRT STATION', 1: 'DISTANCE_MRT'})
df_distance_mrt

,ADDRESS,MRT STATION,DISTANCE_MRT
0,359 WOODLANDS AVENUE 5 SINGAPORE 730359,WOODLANDS MRT STATION,0.4121162706796198 km
1,515A TAMPINES CENTRAL 7 THE PREMIERE @ TAMPINE...,TAMPINES MRT STATION,0.5576863639731215 km
2,246 SIMEI STREET 5 SINGAPORE 520246,SIMEI MRT STATION,0.22495411013969127 km
3,183 JELEBU ROAD SINGAPORE 670183,BUKIT PANJANG MRT STATION,0.19721228372555052 km
4,109 BEDOK NORTH ROAD FENGSHAN ESTATE SINGAPORE...,BEDOK RESERVOIR MRT STATION,0.588027612421757 km
...,...,...,...
9072,222 JURONG EAST STREET 21 SINGAPORE 600222,CHINESE GARDEN MRT STATION,0.38763476453104184 km
9073,93 COMMONWEALTH DRIVE COMMONWEALTH 16 SINGAPOR...,COMMONWEALTH MRT STATION,0.3675665234452749 km
9074,972 HOUGANG STREET 91 SINGAPORE 530972,TONGKANG LRT STATION,1.422161424690531 km
9075,714 CLEMENTI WEST STREET 2 SINGAPORE 120714,CLEMENTI MRT STATION,1.2452723466691529 km


In [22]:
df_distance_mrt.to_csv(filepath + 'distance_mrt.csv', index=False)

In [23]:
hdb = df_resale.merge(geo_edited[geo_edited['INDICATOR'] == 0], how='left', left_on='address', right_on='address')
hdb.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,address,INDICATOR,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,406 ANG MO KIO AVE 10,0.0,406 ANG MO KIO AVENUE 10 SINGAPORE 560406,560406,30288.234663,38229.067463,1.362005,103.853880
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,108 ANG MO KIO AVE 4,0.0,108 ANG MO KIO AVENUE 4 THE ANIMAL DOCTORS SIN...,560108,28518.183285,39217.398915,1.370943,103.837975
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,602 ANG MO KIO AVE 5,0.0,602 ANG MO KIO AVENUE 5 YIO CHU KANG GREEN SIN...,560602,28228.099954,40297.283149,1.380709,103.835368
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,465 ANG MO KIO AVE 10,0.0,465 ANG MO KIO AVENUE 10 TECK GHEE HORIZON SIN...,560465,30657.824693,38693.098657,1.366201,103.857201
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,601 ANG MO KIO AVE 5,0.0,601 ANG MO KIO AVENUE 5 MY FIRST SKOOL SINGAPO...,560601,28201.782487,40334.051212,1.381041,103.835132


In [24]:
hdb = hdb.merge(df_distance_mrt, how='left', left_on='ADDRESS', right_on='ADDRESS')
hdb.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,address,INDICATOR,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE,MRT STATION,DISTANCE_MRT
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,406 ANG MO KIO AVE 10,0.0,406 ANG MO KIO AVENUE 10 SINGAPORE 560406,560406,30288.234663,38229.067463,1.362005,103.853880,ANG MO KIO MRT STATION,1.0002190239027458 km
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,108 ANG MO KIO AVE 4,0.0,108 ANG MO KIO AVENUE 4 THE ANIMAL DOCTORS SIN...,560108,28518.183285,39217.398915,1.370943,103.837975,ANG MO KIO MRT STATION,1.2933927826808795 km
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,602 ANG MO KIO AVE 5,0.0,602 ANG MO KIO AVENUE 5 YIO CHU KANG GREEN SIN...,560602,28228.099954,40297.283149,1.380709,103.835368,YIO CHU KANG MRT STATION,1.0719242691470874 km
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,465 ANG MO KIO AVE 10,0.0,465 ANG MO KIO AVENUE 10 TECK GHEE HORIZON SIN...,560465,30657.824693,38693.098657,1.366201,103.857201,ANG MO KIO MRT STATION,0.9458473209129798 km
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,601 ANG MO KIO AVE 5,0.0,601 ANG MO KIO AVENUE 5 MY FIRST SKOOL SINGAPO...,560601,28201.782487,40334.051212,1.381041,103.835132,YIO CHU KANG MRT STATION,1.094824742018516 km


In [25]:
hdb_final = hdb.drop(columns=['flat_model', 'lease_commence_date', 'remaining_lease', 'address', 'INDICATOR', 'POSTAL', 'X', 'Y', 'LATITUDE', 'LONGITUDE', 'MRT STATION'])
hdb_final.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,resale_price,ADDRESS,DISTANCE_MRT
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,232000.0,406 ANG MO KIO AVENUE 10 SINGAPORE 560406,1.0002190239027458 km
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,250000.0,108 ANG MO KIO AVENUE 4 THE ANIMAL DOCTORS SIN...,1.2933927826808795 km
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,262000.0,602 ANG MO KIO AVENUE 5 YIO CHU KANG GREEN SIN...,1.0719242691470874 km
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,265000.0,465 ANG MO KIO AVENUE 10 TECK GHEE HORIZON SIN...,0.9458473209129798 km
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,265000.0,601 ANG MO KIO AVENUE 5 MY FIRST SKOOL SINGAPO...,1.094824742018516 km


We will drop data which addresses were not found:

In [26]:
hdb_final.dropna(axis=0, inplace=True)

In [27]:
hdb_final['DISTANCE_MRT'] = hdb_final['DISTANCE_MRT'].apply(lambda x: round(x.km, 3))
hdb_final.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,resale_price,ADDRESS,DISTANCE_MRT
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,232000.0,406 ANG MO KIO AVENUE 10 SINGAPORE 560406,1.000
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,250000.0,108 ANG MO KIO AVENUE 4 THE ANIMAL DOCTORS SIN...,1.293
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,262000.0,602 ANG MO KIO AVENUE 5 YIO CHU KANG GREEN SIN...,1.072
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,265000.0,465 ANG MO KIO AVENUE 10 TECK GHEE HORIZON SIN...,0.946
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,265000.0,601 ANG MO KIO AVENUE 5 MY FIRST SKOOL SINGAPO...,1.095


I have decided to also include the distance from City Hall MRT which is the city centre of Singapore. The distance is calculated by using the find_nearest() function defined earlier:

In [29]:
citycentre_data = [['cityhall', 1.29317576, 103.8525073]]
citycentre = pd.DataFrame(citycentre_data, columns=['address', 'LATITUDE', 'LONGITUDE'])
citycentre

,address,LATITUDE,LONGITUDE
0,cityhall,1.293176,103.852507


In [30]:
distance_citycentre = find_nearest(geo_add_lat_long, citycentre)

Streaming output truncated to the last 5000 lines.
4078 - 349 CORPORATION DRIVE SINGAPORE 610349 : ('cityhall', Distance(15.699916150176438))
4079 - 816 JURONG WEST STREET 81 SINGAPORE 640816 : ('cityhall', Distance(18.302844128162373))
4080 - 242 PASIR RIS STREET 21 SINGAPORE 510242 : ('cityhall', Distance(15.000701065844595))
4081 - 5 LORONG LEW LIAN LEW LIAN GARDENS SINGAPORE 530005 : ('cityhall', Distance(6.845369999550142))
4082 - 111 COMMONWEALTH CRESCENT COMMONWEALTH NEIGHBOURHOOD POLICE POST SINGAPORE 140111 : ('cityhall', Distance(6.204916794503689))
4083 - 326B ANCHORVALE ROAD ANCHORVALE HORIZON SINGAPORE 542326 : ('cityhall', Distance(11.9650405338138))
4084 - 10 JOO SENG ROAD SINGAPORE 360010 : ('cityhall', Distance(5.52029846720291))
4085 - 370 WOODLANDS AVENUE 1 SINGAPORE 730370 : ('cityhall', Distance(17.029464518064465))
4086 - 142 MARSILING ROAD SINGAPORE 730142 : ('cityhall', Distance(17.986558087357555))
4087 - 194 PASIR RIS STREET 12 : ('cityhall', Distance(14.38991

In [31]:
df_distance_citycentre = pd.DataFrame(distance_citycentre).T.reset_index()
df_distance_citycentre

,index,0,1
0,359 WOODLANDS AVENUE 5 SINGAPORE 730359,cityhall,17.340811269190347 km
1,515A TAMPINES CENTRAL 7 THE PREMIERE @ TAMPINE...,cityhall,11.878165269856177 km
2,246 SIMEI STREET 5 SINGAPORE 520246,cityhall,12.685049199925723 km
3,183 JELEBU ROAD SINGAPORE 670183,cityhall,13.827982365181429 km
4,109 BEDOK NORTH ROAD FENGSHAN ESTATE SINGAPORE...,cityhall,10.130746500409261 km
...,...,...,...
9072,222 JURONG EAST STREET 21 SINGAPORE 600222,cityhall,14.020625077787805 km
9073,93 COMMONWEALTH DRIVE COMMONWEALTH 16 SINGAPOR...,cityhall,5.98629634647388 km
9074,972 HOUGANG STREET 91 SINGAPORE 530972,cityhall,9.869265737342385 km
9075,714 CLEMENTI WEST STREET 2 SINGAPORE 120714,cityhall,10.182651295779072 km


In [32]:
df_distance_citycentre = df_distance_citycentre.rename(columns={'index':'ADDRESS', 0: 'LOCATION', 1: 'DISTANCE_CITYCENTRE'})
df_distance_citycentre

,ADDRESS,LOCATION,DISTANCE_CITYCENTRE
0,359 WOODLANDS AVENUE 5 SINGAPORE 730359,cityhall,17.340811269190347 km
1,515A TAMPINES CENTRAL 7 THE PREMIERE @ TAMPINE...,cityhall,11.878165269856177 km
2,246 SIMEI STREET 5 SINGAPORE 520246,cityhall,12.685049199925723 km
3,183 JELEBU ROAD SINGAPORE 670183,cityhall,13.827982365181429 km
4,109 BEDOK NORTH ROAD FENGSHAN ESTATE SINGAPORE...,cityhall,10.130746500409261 km
...,...,...,...
9072,222 JURONG EAST STREET 21 SINGAPORE 600222,cityhall,14.020625077787805 km
9073,93 COMMONWEALTH DRIVE COMMONWEALTH 16 SINGAPOR...,cityhall,5.98629634647388 km
9074,972 HOUGANG STREET 91 SINGAPORE 530972,cityhall,9.869265737342385 km
9075,714 CLEMENTI WEST STREET 2 SINGAPORE 120714,cityhall,10.182651295779072 km


In [33]:
df_distance_citycentre.to_csv(filepath + 'distance_citycentre.csv', index=False)

In [34]:
hdb_final = hdb_final.merge(df_distance_citycentre, how='left', left_on='ADDRESS', right_on='ADDRESS')
hdb_final.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,resale_price,ADDRESS,DISTANCE_MRT,LOCATION,DISTANCE_CITYCENTRE
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,232000.0,406 ANG MO KIO AVENUE 10 SINGAPORE 560406,1.000,cityhall,7.612266338225603 km
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,250000.0,108 ANG MO KIO AVENUE 4 THE ANIMAL DOCTORS SIN...,1.293,cityhall,8.749844563771587 km
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,262000.0,602 ANG MO KIO AVENUE 5 YIO CHU KANG GREEN SIN...,1.072,cityhall,9.865110551058121 km
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,265000.0,465 ANG MO KIO AVENUE 10 TECK GHEE HORIZON SIN...,0.946,cityhall,8.091639228670868 km
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,265000.0,601 ANG MO KIO AVENUE 5 MY FIRST SKOOL SINGAPO...,1.095,cityhall,9.906290988634552 km


In [35]:
hdb_final.drop(columns=['LOCATION'], inplace=True)
hdb_final.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,resale_price,ADDRESS,DISTANCE_MRT,DISTANCE_CITYCENTRE
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,232000.0,406 ANG MO KIO AVENUE 10 SINGAPORE 560406,1.000,7.612266338225603 km
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,250000.0,108 ANG MO KIO AVENUE 4 THE ANIMAL DOCTORS SIN...,1.293,8.749844563771587 km
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,262000.0,602 ANG MO KIO AVENUE 5 YIO CHU KANG GREEN SIN...,1.072,9.865110551058121 km
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,265000.0,465 ANG MO KIO AVENUE 10 TECK GHEE HORIZON SIN...,0.946,8.091639228670868 km
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,265000.0,601 ANG MO KIO AVENUE 5 MY FIRST SKOOL SINGAPO...,1.095,9.906290988634552 km


In [36]:
hdb_final['DISTANCE_CITYCENTRE'] = hdb_final['DISTANCE_CITYCENTRE'].apply(lambda x: round(x.km, 3))
hdb_final.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,resale_price,ADDRESS,DISTANCE_MRT,DISTANCE_CITYCENTRE
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,232000.0,406 ANG MO KIO AVENUE 10 SINGAPORE 560406,1.000,7.612
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,250000.0,108 ANG MO KIO AVENUE 4 THE ANIMAL DOCTORS SIN...,1.293,8.750
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,262000.0,602 ANG MO KIO AVENUE 5 YIO CHU KANG GREEN SIN...,1.072,9.865
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,265000.0,465 ANG MO KIO AVENUE 10 TECK GHEE HORIZON SIN...,0.946,8.092
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,265000.0,601 ANG MO KIO AVENUE 5 MY FIRST SKOOL SINGAPO...,1.095,9.906


In [37]:
hdb_final.to_csv(filepath + 'hdb_dataset.csv', index=False)

## Data Preparation

We will first drop the columns 'month', 'block', 'street_name', 'ADDRESS', since they are not useful for predicting the resale price:

In [38]:
path = '/content/drive/MyDrive/Colab Notebooks/Housing/hdb_dataset.csv'
df_final = pd.read_csv(path)
df_final.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,resale_price,ADDRESS,DISTANCE_MRT,DISTANCE_CITYCENTRE
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,232000.0,406 ANG MO KIO AVENUE 10 SINGAPORE 560406,1.000,7.612
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,250000.0,108 ANG MO KIO AVENUE 4 THE ANIMAL DOCTORS SIN...,1.293,8.750
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,262000.0,602 ANG MO KIO AVENUE 5 YIO CHU KANG GREEN SIN...,1.072,9.865
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,265000.0,465 ANG MO KIO AVENUE 10 TECK GHEE HORIZON SIN...,0.946,8.092
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,265000.0,601 ANG MO KIO AVENUE 5 MY FIRST SKOOL SINGAPO...,1.095,9.906


In [39]:
df_final.drop(columns=['month', 'block', 'street_name', 'ADDRESS'], inplace=True)
df_final.head()

,town,flat_type,storey_range,floor_area_sqm,resale_price,DISTANCE_MRT,DISTANCE_CITYCENTRE
0,ANG MO KIO,2 ROOM,10 TO 12,44.0,232000.0,1.000,7.612
1,ANG MO KIO,3 ROOM,01 TO 03,67.0,250000.0,1.293,8.750
2,ANG MO KIO,3 ROOM,01 TO 03,67.0,262000.0,1.072,9.865
3,ANG MO KIO,3 ROOM,04 TO 06,68.0,265000.0,0.946,8.092
4,ANG MO KIO,3 ROOM,01 TO 03,67.0,265000.0,1.095,9.906


In [40]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df_final, random_state=42, shuffle=True, test_size=0.2)

In [41]:
X_train = df_train.drop(columns=['resale_price'])
y_train = df_train[['resale_price']]

In [42]:
X_train.shape, y_train.shape

((83887, 6), (83887, 1))

The predictors are split into numerical and categorical variables. They are then passed through the ColumnTransformer(), which it will preprocess the numerical and categorical variables separately, and then concetenate them. The numerical variables are scaled by using StandardScaler, whereas the categorical variables are being one-hot encoded:

In [43]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

hdb_num = X_train.select_dtypes(exclude=['object'])
hdb_cat = X_train.select_dtypes(include=['object'])

num_attribs = list(hdb_num)
cat_attribs = list(hdb_cat)

transform_pipeline = ColumnTransformer([('num', StandardScaler(), num_attribs), 
                                        ('cat', OneHotEncoder(), cat_attribs)])

X_train_prepared = transform_pipeline.fit_transform(X_train).toarray()

In [44]:
X_train_prepared[0]

array([ 0.50328456, -1.02739438,  0.34190807,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ])

In [45]:
y_train_prepared = y_train.values

In [46]:
X_train_prepared.shape, y_train_prepared.shape

((83887, 53), (83887, 1))

## Model Building

Finally, our dataset is ready for training. We will further split it into training and validation dataset:

In [47]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train_prepared, y_train_prepared, test_size=0.2, random_state=42)

In [48]:
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((67109, 53), (16778, 53), (67109, 1), (16778, 1))

### Linear Regression

In [81]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [82]:
from sklearn.metrics import mean_squared_error

y_pred = lin_reg.predict(X_valid)
lin_mse = mean_squared_error(y_valid, y_pred)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

77101.41063314029

Let's see we can do better with added polynomial features, plus some regularization:

### Lasso Regression

In [51]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline

model = Pipeline([('poly_features', PolynomialFeatures(degree=2, include_bias=False)), 
                  ('lasso', Lasso(alpha=1, tol=1e-3, random_state=42))])
model.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 94918678654677.5, tolerance: 1647754287752.351
  positive)


Pipeline(memory=None,
         steps=[('poly_features',
                 PolynomialFeatures(degree=2, include_bias=False,
                                    interaction_only=False, order='C')),
                ('lasso',
                 Lasso(alpha=1, copy_X=True, fit_intercept=True, max_iter=1000,
                       normalize=False, positive=False, precompute=False,
                       random_state=42, selection='cyclic', tol=0.001,
                       warm_start=False))],
         verbose=False)

In [52]:
y_pred = model.predict(X_valid)
lasso_mse = mean_squared_error(y_valid, y_pred)
lasso_rmse = np.sqrt(lasso_mse)
lasso_rmse

61428.090260272984

### Decision Tree

In [53]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(X_train, y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=42, splitter='best')

In [54]:
y_pred = tree_reg.predict(X_valid)
tree_mse = mean_squared_error(y_valid, y_pred)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

42850.8474000905

In [55]:
y_pred = tree_reg.predict(X_train)
tree_mse = mean_squared_error(y_train, y_pred)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

13850.84672617378

Looks like the model has overfitted the training data. Let's try using an ensemble of Decision Tree Regressor to trade a higher bias for lower variance.

### Random Forest

In [56]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=100, random_state=42)
forest_reg.fit(X_train, y_train.ravel())

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [57]:
y_pred = forest_reg.predict(X_valid)
forest_mse = mean_squared_error(y_valid, y_pred)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

34651.23371316893

In [58]:
y_pred = forest_reg.predict(X_train)
forest_mse = mean_squared_error(y_train, y_pred)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

17354.372910674007

It is better than the Decision Tree, but still overfits the training data. We will use GridSearchCV to tune the hyperparameters:

In [59]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [30, 40, 50], 'max_features': [30, 40, 50]},
    {'bootstrap': [False], 'n_estimators': [30, 50], 'max_features': [30, 40, 50]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(forest_reg, param_grid, cv=3, scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(X_train, y_train.ravel())

GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jobs

In [60]:
forest_reg = grid_search.best_estimator_
y_pred = forest_reg.predict(X_valid)
forest_mse = mean_squared_error(y_valid, y_pred)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

34179.7951307855

In [61]:
y_pred = forest_reg.predict(X_train)
forest_mse = mean_squared_error(y_train, y_pred)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

17332.803305671318

After tuning the hyperparameters, the model still overfits the training data. Let's try other ensemble methods.

### Gradient Boosting

In [62]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(n_estimators=500, learning_rate=0.05, random_state=42)
gbrt.fit(X_train, y_train.ravel())

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.05, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=500,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=42, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [63]:
y_pred = gbrt.predict(X_valid)
gbrt_mse = mean_squared_error(y_valid, y_pred)
gbrt_rmse = np.sqrt(gbrt_mse)
gbrt_rmse

57963.4569677588

In [64]:
y_pred = gbrt.predict(X_train)
gbrt_mse = mean_squared_error(y_train, y_pred)
gbrt_rmse = np.sqrt(gbrt_mse)
gbrt_rmse

56663.28376587096

### XGBoost

In [97]:
import xgboost

xgb_reg = xgboost.XGBRegressor(n_estimators=500, learning_rate=0.05, random_state=42)
xgb_reg.fit(X_train, y_train.ravel())

[13:15:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=500,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [98]:
y_pred = xgb_reg.predict(X_valid)
xgb_mse = mean_squared_error(y_valid, y_pred)
xgb_rmse = np.sqrt(xgb_mse)
xgb_rmse

57969.765595731646

In [99]:
y_pred = xgb_reg.predict(X_train)
xgb_mse = mean_squared_error(y_train, y_pred)
xgb_rmse = np.sqrt(xgb_mse)
xgb_rmse

56775.40697888984

### Deep Neural Networks (DNNs)

Let's try implementing DNNs with Keras to predict the resale flat prices:

In [83]:
import tensorflow as tf
from tensorflow import keras

np.random.seed(42)
tf.random.set_seed(42)
keras.backend.clear_session()

In [84]:
model = keras.models.Sequential()
model.add(keras.layers.Input(shape=X_train.shape[-1]))
for _ in range(20):
  model.add(keras.layers.Dense(100, activation='elu', kernel_initializer='he_normal'))
model.add(keras.layers.Dense(1))

In [85]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               5400      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_6 (Dense)              (None, 100)               1

In [86]:
model.compile(loss='mse', optimizer='Adam')
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint('hdb_nn_model.h5', monitor='val_loss', save_best_only=True)
callbacks = [early_stopping_cb, model_checkpoint_cb]
model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=callbacks)

Epoch 1/100
2098/2098 [==============================] - 13s 6ms/step - loss: 7655622144.0000 - val_loss: 4189966848.0000
Epoch 2/100
2098/2098 [==============================] - 12s 6ms/step - loss: 4350568960.0000 - val_loss: 3895985920.0000
Epoch 3/100
2098/2098 [==============================] - 12s 6ms/step - loss: 3893948416.0000 - val_loss: 3442310912.0000
Epoch 4/100
2098/2098 [==============================] - 12s 6ms/step - loss: 3578223616.0000 - val_loss: 3372577792.0000
Epoch 5/100
2098/2098 [==============================] - 12s 6ms/step - loss: 3387738624.0000 - val_loss: 3088817408.0000
Epoch 6/100
2098/2098 [==============================] - 12s 6ms/step - loss: 3229887744.0000 - val_loss: 3027268096.0000
Epoch 7/100
2098/2098 [==============================] - 12s 6ms/step - loss: 2966417664.0000 - val_loss: 2843781632.0000
Epoch 8/100
2098/2098 [==============================] - 11s 5ms/step - loss: 2880650752.0000 - val_loss: 2835031296.0000
Epoch 9/100
2098/2098 [=

In [100]:
model = keras.models.load_model('hdb_nn_model.h5')
y_pred = model.predict(X_valid)
ann_mse = mean_squared_error(y_valid, y_pred)
ann_rmse = np.sqrt(ann_mse)
ann_rmse

42203.3035343964

In [101]:
y_pred = model.predict(X_train)
ann_mse = mean_squared_error(y_train, y_pred)
ann_rmse = np.sqrt(ann_mse)
ann_rmse

39283.426700442986

The DNN model performs better than the previous models in terms of lower RMSE on the validation dataset, and also does not overfit the training dataset.

We will try adding BatchNormalization to see if it improves the model:

In [70]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [71]:
model = keras.models.Sequential()
model.add(keras.layers.Input(shape=X_train.shape[-1]))
model.add(keras.layers.BatchNormalization())
for _ in range(20):
  model.add(keras.layers.Dense(100, kernel_initializer='he_normal'))
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.Activation('elu'))
model.add(keras.layers.Dense(1))

In [72]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 53)                212       
_________________________________________________________________
dense (Dense)                (None, 100)               5400      
_________________________________________________________________
batch_normalization_1 (Batch (None, 100)               400       
_________________________________________________________________
activation (Activation)      (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
activation_1 (Activation)    (None, 100)               0

It makes sense to first scale the label data (i.e., y_train), or else it will explode the weights connected to the output layer:

In [73]:
scaler = StandardScaler()
y_train_scaled = scaler.fit_transform(y_train)
y_valid_scaled = scaler.transform(y_valid)

In [74]:
optimizer = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='mse', optimizer=optimizer)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint('hdb_nn_model_v1.h5', monitor='val_loss', save_best_only=True)
callbacks = [early_stopping_cb, model_checkpoint_cb]
model.fit(X_train, y_train_scaled, epochs=100, validation_data=(X_valid, y_valid_scaled), callbacks=callbacks)

Epoch 1/100
2098/2098 [==============================] - 25s 10ms/step - loss: 0.2945 - val_loss: 0.3014
Epoch 2/100
2098/2098 [==============================] - 21s 10ms/step - loss: 0.2456 - val_loss: 0.5939
Epoch 3/100
2098/2098 [==============================] - 21s 10ms/step - loss: 0.2327 - val_loss: 0.3508
Epoch 4/100
2098/2098 [==============================] - 21s 10ms/step - loss: 0.2244 - val_loss: 1.0801
Epoch 5/100
2098/2098 [==============================] - 21s 10ms/step - loss: 0.2170 - val_loss: 0.5890
Epoch 6/100
2098/2098 [==============================] - 21s 10ms/step - loss: 0.2080 - val_loss: 0.7070
Epoch 7/100
2098/2098 [==============================] - 21s 10ms/step - loss: 0.1971 - val_loss: 0.5031
Epoch 8/100
2098/2098 [==============================] - 21s 10ms/step - loss: 0.1907 - val_loss: 0.2314
Epoch 9/100
2098/2098 [==============================] - 21s 10ms/step - loss: 0.1853 - val_loss: 9.5225
Epoch 10/100
2098/2098 [==============================]

In [102]:
model = keras.models.load_model('hdb_nn_model_v1.h5')
y_pred_scaled = model.predict(X_valid)
y_pred = scaler.inverse_transform(y_pred_scaled)
ann_mse = mean_squared_error(y_valid, y_pred)
ann_rmse = np.sqrt(ann_mse)
ann_rmse

57622.00829246267

In [103]:
y_pred_scaled = model.predict(X_train)
y_pred = scaler.inverse_transform(y_pred_scaled)
ann_mse = mean_squared_error(y_train, y_pred)
ann_rmse = np.sqrt(ann_mse)
ann_rmse

56652.93837007423

The model has become worse when using BatchNormalization(), let's use SELU instead, and making necessary adjustments to ensure the network self-normalizes:

In [88]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [89]:
model = keras.models.Sequential()
model.add(keras.layers.Input(shape=X_train.shape[-1]))
for _ in range(20):
  model.add(keras.layers.Dense(100, activation='selu', kernel_initializer='lecun_normal'))
model.add(keras.layers.Dense(1))

In [90]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               5400      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_6 (Dense)              (None, 100)               1

In [91]:
model.compile(loss='mse', optimizer='Adam')
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint('hdb_nn_model_v2.h5', monitor='val_loss', save_best_only=True)
callbacks = [early_stopping_cb, model_checkpoint_cb]
model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=callbacks)

Epoch 1/100
2098/2098 [==============================] - 13s 6ms/step - loss: 14049262592.0000 - val_loss: 4714802176.0000
Epoch 2/100
2098/2098 [==============================] - 12s 6ms/step - loss: 5052487680.0000 - val_loss: 5254250496.0000
Epoch 3/100
2098/2098 [==============================] - 12s 6ms/step - loss: 4913239552.0000 - val_loss: 4463831040.0000
Epoch 4/100
2098/2098 [==============================] - 12s 6ms/step - loss: 4555335168.0000 - val_loss: 4304059392.0000
Epoch 5/100
2098/2098 [==============================] - 12s 6ms/step - loss: 4282134272.0000 - val_loss: 4300420096.0000
Epoch 6/100
2098/2098 [==============================] - 12s 6ms/step - loss: 4037105152.0000 - val_loss: 4055691264.0000
Epoch 7/100
2098/2098 [==============================] - 12s 6ms/step - loss: 3708185344.0000 - val_loss: 3615235072.0000
Epoch 8/100
2098/2098 [==============================] - 12s 6ms/step - loss: 3749037056.0000 - val_loss: 3355481088.0000
Epoch 9/100
2098/2098 [

In [104]:
model = keras.models.load_model('hdb_nn_model_v2.h5')
y_pred = model.predict(X_valid)
ann_mse = mean_squared_error(y_valid, y_pred)
ann_rmse = np.sqrt(ann_mse)
ann_rmse

41792.014726080495

In [105]:
y_pred = model.predict(X_train)
ann_mse = mean_squared_error(y_train, y_pred)
ann_rmse = np.sqrt(ann_mse)
ann_rmse

39644.04882571963

By using SELU activation, we managed to improve the original model slightly and it is the best model so far.

We can further experiment the dataset with different configuration of the DNNs (e.g., different number of hidden layers with different number of neurons), or by changing the optimizer (e.g., RMSProp, Nadam), or even introducing a learning rate scheduler (e.g., OneCycle). 

To conclude this part of the project, there are still room for improvement in terms of selecting the right Machine Learning model. We can do so by filtering out the top performance models, and by further tweaking their hyperparameters to achieve a better accuracy on predicting the resale price.